In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from typing import Optional
from scipy.stats import anderson, shapiro
import statsmodels.api as sm
from scipy.special import ive , iv # check if it's just modified bessel function of first kind
from scipy.optimize import fmin, fmin_powell, minimize
from matplotlib import cm
from matplotlib.ticker import LinearLocator

In [2]:
df = pd.read_csv('calibration_data_derivatives.csv')
df = df.drop(['Unnamed: 0'],axis=1)
T = df['T'].to_numpy()

In [3]:
df.head()

,T,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0.0000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
1,0.0004,20.009467,20.042233,20.054410,20.022644,20.019337,20.029027,19.994295,20.022623,19.969734,...,19.979903,20.033890,20.084984,20.030623,20.075743,20.071736,19.958307,20.073213,20.000348,20.030370
2,0.0008,20.180659,20.034681,20.073286,20.071441,20.084694,20.072978,20.044870,20.060890,20.013023,...,19.945434,20.115835,20.151885,20.033409,20.139381,20.133000,20.002493,20.147919,20.031410,20.073345
3,0.0012,20.250422,20.058778,20.108238,20.105163,20.124961,20.123804,20.108782,20.123590,20.001301,...,20.054304,20.108343,20.169604,20.058534,20.168256,20.200677,20.061395,20.216549,20.085283,20.083917
4,0.0016,20.356734,20.099250,20.144123,20.196775,20.134417,20.205258,20.159322,20.109303,20.087247,...,20.091437,20.173392,20.220396,20.032751,20.258030,20.320258,20.100482,20.251627,20.066172,20.143806


In [4]:
class CIR:
    def __init__(self,a:float,b:float,sigma:float):
        self.a = a
        self.b = b 
        self.sigma = sigma 

    def post_init(self):
        if 2 * self.a * self.b > self.sigma ** 2:
            raise ValueError("For positive rate, 2ab has to be greater than or equal to c^2.")
        return None

    # generates data from the given configuration
    def generate_eulerCIR(self,S0, T, random_state:Optional[int] = None):
        a,b,sigma = self.a, self.b, self.sigma
        S = [S0]
        
        for i in range(len(T)-1):
            delT = T[i+1]-T[i]
            if random_state:
                np.random.seed(random_state+i)
                
            S_new = S[-1] + a*(b-S[-1])*delT + sigma*np.sqrt(max(S[-1],0)*delT)*np.random.normal()
            S.append(S_new)

        return np.array(S)

    def generate_PCeulerCIR(self, S0, T, random_state: Optional[int] = None):
        a,b,sigma = self.a, self.b, self.sigma
        S = [S0]

        for i in range(len(T)-1):
            delT = T[i+1]-T[i]
            if random_state:
                np.random.seed(random_state+i)
                
            S_star = S[-1] + a*(b-S[-1])*delT + sigma*np.sqrt(max(S[-1],0)*delT)*np.random.normal()
            S_new = S_star + (a*(S[-1]-S_star)*delT)/2

            S.append(S_new)

        return np.array(S)  

    def generate_milsteinCIR(self, S0, T, random_state:Optional[int]=None):
        a,b,sigma = self.a, self.b, self.sigma
        S = [S0]

        for i in range(len(T)-1):
            delT = T[i+1]-T[i]
            if random_state:
                np.random.seed(random_state+i)

            z = np.random.normal() 
            S_new = S[-1] + (a*b*delT) - (a*delT*S[-1]) + (sigma*np.sqrt(S[-1]*delT)*z) + ((sigma**2 * delT * (z**2 -1))/4)    
            S.append(S_new)

        return np.array(S) 
    

## Estimate Parameters

### 1. OLS

In [5]:
class Solver:
    def OLS_statsmodel(self,X,y):
        # solve using stats model sm library implementation
        lr = sm.OLS(y, X).fit()
        print(lr.summary())
        
        thetahat = lr.params
        yhat = X@thetahat
        yhat = np.squeeze(yhat)
        error = y-yhat

        n = len(yhat)
        p = X.shape[1]

        varhat = error.T @ error / (n-p)
        sigmahat = np.sqrt(varhat)

        return [thetahat,sigmahat]    


    def OLS(self,X,y, estimate_metrics=False):
        ''' 
        Closed form solution for Linear Regression
        '''
        # Handtyped implementation
        thetahat = np.linalg.inv(X.T @ X) @ X.T @ y
        yhat = X@thetahat
        error = y-yhat
        
        n = len(yhat)
        p = X.shape[1]

        varhat = error.T @ error / (n-p)
        
        sigmahat = np.sqrt(varhat)

        if estimate_metrics:
            self.thetahat_cov = varhat*np.linalg.inv(X.T @ X)
            self.errorBias = np.mean(error)
            # anderson-darling test for checking normality condition for error terms
            result = anderson(error)
            stat, p = shapiro(error)
            alpha = 0.05 
            flag= True

            for i in range(len(result.critical_values)):
                sl, cv = result.significance_level[i], result.critical_values[i]
                if result.statistic < result.critical_values[i]:
                    pass
                else:
                    print('AD Test: Error does not look normal (reject H0)')
                    flag = False
                    break

            if flag: 
                print('AD Test: Error looks normal (fail to reject H0)')
                self.isWhite = True 
            else:
                self.isWhite = False

            if p > alpha:
                print('Shapiro Test: Sample looks Gaussian (fail to reject H0)')
            else:
                print('Shapiro Test: Sample does not look Gaussian (reject H0)')

        return [thetahat,sigmahat]

In [6]:
def estimateParams_euler(T, cir, estimate_metrics = False):
    delT = np.diff(T)
    y = np.diff(cir)/np.sqrt(cir[:-1])
    x1 = delT/np.sqrt(cir[:-1])
    x2 = -np.sqrt(cir[:-1])*delT

    X = np.array((x1,x2)).T
    solver = Solver()
    thetahat, sig = solver.OLS(X,y, estimate_metrics)


    abhat, ahat = thetahat
    bhat = abhat/ahat
    sigmahat = sig/np.sqrt(delT[0])

    if estimate_metrics: 
        print("Mean of Errors: ", solver.errorBias)
        print("Covariance matrix for parameter estimates: ", solver.thetahat_cov)
        # solver.OLS_statsmodel(X,y)
    return ahat,bhat, sigmahat

### 2. MLE

In [7]:
def negloglik(x,T,cir):
    ''' 
    x is [a,b,sigma] 
    Evalues Negative Log Likelihood for parameters contained in x on cir. 
    '''
    dataF = cir[1:]
    dataL = cir[:-1]
    Nobs = len(cir)
    Timestep = (T[1]-T[0])
    a,b,sigma = x

    c = 2*a/((sigma**2) * (1-np.exp(-a*Timestep)))
    q = (2*a*b/(sigma**2)) - 1 
    u = c*dataL* np.exp(-a*Timestep)
    v = c*dataF
    z = 2* np.sqrt(u*v)  
    bf = ive(q,z)
    lnL = -(Nobs-1)*np.log(c) + np.sum(u+v - 0.5*q*np.log(v/u) - np.log(bf) - z)

    return lnL

In [13]:
def calibrate_dataOLS(df: pd.DataFrame,estimate_metrics = False):
    ''' 
    Calibrate the model parameters using OLS method
    '''
    ahatvec = []
    bhatvec = []
    sigmahatvec = []

    for i in tqdm(range(100)):
        cir = df[str(i)].to_numpy()
        guess = estimateParams_euler(T,cir, estimate_metrics)
        
        ahatvec.append(guess[0])
        bhatvec.append(guess[1])
        sigmahatvec.append(guess[2])  

        if estimate_metrics:
            print("")

    ahatmean = np.mean(ahatvec)
    bhatmean = np.mean(bhatvec)
    sigmahatvec = np.mean(sigmahatvec)

    return ahatmean, bhatmean, sigmahatvec

In [41]:
def calibrate_dataMLE(df: pd.DataFrame,method = "Nelder-Mead"):
    ''' 
    Calibrate the model parameters using MLE method. The initial guess is OLS estimate
    '''
    ahatvec = []
    bhatvec = []
    sigmahatvec = []
    nit = []

    for i in tqdm(range(100)):
        cir = df[str(i)].to_numpy()
        guess = estimateParams_euler(T,cir)
        xopt = minimize(fun=negloglik,args = (T,cir), x0=guess, method=method) 
        nit.append(xopt.nit)
        
        ahatvec.append(xopt.x[0])
        bhatvec.append(xopt.x[1])
        sigmahatvec.append(xopt.x[2])  

    ahatmean = np.mean(ahatvec)
    bhatmean = np.mean(bhatvec)
    sigmahatvec = np.mean(sigmahatvec)
    print("No of iter: ", np.mean(nit))

    return ahatmean, bhatmean, sigmahatvec

In [10]:
ols = calibrate_dataOLS(df)
ols

100%|██████████| 100/100 [00:00<00:00, 1644.20it/s]


(0.9098468894490687, 135.09397038781802, 0.399912442590426)

In [18]:
a,b,sigma = calibrate_dataMLE(df)

100%|██████████| 100/100 [00:21<00:00,  4.71it/s]


In [24]:
calibrate_dataMLE(df, method = 'bfgs')

100%|██████████| 100/100 [00:33<00:00,  2.95it/s]


(0.9098549658090235, 135.09396824764636, 0.39990949958243)

In [23]:
calibrate_dataMLE(df,method = 'L-BFGS-B')

100%|██████████| 100/100 [00:06<00:00, 14.30it/s]


(0.9069713800379191, 135.06920819671774, 0.3425973787638769)

## Option Pricing

In [49]:
# estimated parameters from the given data through MLE 
a,b,sigma

(0.9100101993332262, 135.09418157986403, 0.39990734424709956)

In [50]:
calibrated_model = CIR(a,b,sigma)

In [60]:
# up and out putoption
def priceOption(S_b,K, cirmodel: CIR, strategy = "PC"):
    a,b,sigma = cirmodel.a, cirmodel.b, cirmodel.sigma
    r = 0.1 # discount rate
    tau = 4 # time to expiry
    S0 = 20 # starting price
    delT = 1/12 
    n = 100000 # realizations

    T = np.arange(0,tau+0.01,1/12)

    payoff_uo =[]
    payoff_ui = []
    payoff_p = []

##################################    
    if strategy.lower() == 'pc':
        for j in range(n):
            option_activity = S0<S_b # up and put option activity
            S = [S0] # spot price today

            for i in range(len(T)-1):
                S_star = S[-1] + a*(b-S[-1])*delT + sigma*np.sqrt(max(S[-1],0)*delT)*np.random.normal()
                S_new = S_star + (a*(S[-1]-S_star)*delT)/2
                S.append(S_new)

                if S_new > S_b:
                    option_activity = False 
                
            
            if option_activity: # up and out is active, up and in is inactive
                payoff_uo.append(max(K-S[-1],0))
                payoff_ui.append(0) 

            else:  # up and out is inactive, up and in is active
                payoff_uo.append(0)
                payoff_ui.append(max(K-S[-1],0))

            payoff_p.append(max(K-S[-1],0))     



####################################
    elif strategy.lower() == 'milstein':
        for j in range(n):
            option_activity = S0<S_b # up and put option activity
            S = [S0] # spot price today

            for i in range(len(T)-1):
                z = np.random.normal() 
                S_new = S[-1] + (a*b*delT) - (a*delT*S[-1]) + (sigma*np.sqrt(S[-1]*delT)*z) + ((sigma**2 * delT * (z**2 -1))/4)    
                S.append(S_new)

                if S_new > S_b:
                    option_activity = False 
                
            
            if option_activity: # up and out is active, up and in is inactive
                payoff_uo.append(max(K-S[-1],0))
                payoff_ui.append(0) 

            else:  # up and out is inactive, up and in is active
                payoff_uo.append(0)
                payoff_ui.append(max(K-S[-1],0))

            payoff_p.append(max(K-S[-1],0)) 


#####################################
    else:
        if strategy.lower() == 'euler':
            pass 
        else:
            print("Discretization Strategy not found. Using Euler Maurayana method")

        for j in range(n):
            option_activity = S0<S_b # up and put option activity
            S = [S0] # spot price today

            for i in range(len(T)-1):
                S_new = S[-1] + a*(b-S[-1])*delT + sigma*np.sqrt(max(S[-1],0)*delT)*np.random.normal()
                S.append(S_new)

                if S_new > S_b:
                    option_activity = False 
                
            
            if option_activity: # up and out is active, up and in is inactive
                payoff_uo.append(max(K-S[-1],0))
                payoff_ui.append(0) 

            else:  # up and out is inactive, up and in is active
                payoff_uo.append(0)
                payoff_ui.append(max(K-S[-1],0))

            payoff_p.append(max(K-S[-1],0)) 
        

########################## 
    payoff_uo = np.array(payoff_uo)
    payoff_ui = np.array(payoff_ui)
    payoff_p = np.array(payoff_p)

    uo,ui,p = [np.mean(payoff_uo),np.mean(payoff_ui),np.mean(payoff_p)]
    
    option_price = [uo*np.exp(-r*tau),ui*np.exp(-r*tau),p*np.exp(-r*tau) ]
        
    return option_price  

In [61]:
S_b = [140,140,137,139,150,145,132]
K = [145,150,137,139,200,145,132]

In [73]:
df = pd.DataFrame()

In [63]:
uo = []
ui = []
p = []
for i in tqdm(range(len(K))):
    price = priceOption(S_b[i],K[i],calibrated_model,strategy="pc")
    uo.append(price[0])
    ui.append(price[1])
    p.append(price[2])

100%|██████████| 7/7 [02:44<00:00, 23.54s/it]


In [74]:
df["S_b"] = S_b
df["K"] = K 
df["Up and Out Put"] = uo 
df["Up and In Put"] = ui 
df['Put'] = p 

In [75]:
# note the parity
df

,S_b,K,Up and Out Put,Up and In Put,Put
0,140,145,8.606491,0.064784,8.671275
1,140,150,11.892999,0.129109,12.022108
2,137,137,3.325484,0.065847,3.391331
3,139,139,4.654781,0.022847,4.677628
4,150,200,45.547866,0.000000,45.547866
5,145,145,8.673678,0.000178,8.673855
6,132,132,0.699778,0.172378,0.872155


## Ranking Discretization Methods

In [66]:
def exp_stockprice(cirmodel:CIR, S0, t):
    a,b,sigma = cirmodel.a, cirmodel.b, cirmodel.sigma 

    return S0*np.exp(-a*t) + b*(1-np.exp(-a*t))

In [77]:
# expected stock price value at time t using probability
exp_stockprice(calibrated_model,50,4)

132.86035099173992

In [74]:
time = np.arange(0,4.01,1/12)

In [81]:
# expected stock price value at time t using Monte Carlo Simulation
n=10000

sp_euler = []
sp_pc = []
sp_mil = []
for i in range(n):
    sp_euler.append(calibrated_model.generate_eulerCIR(50,time)[-1])
    sp_pc.append(calibrated_model.generate_PCeulerCIR(50,time)[-1])
    sp_mil.append(calibrated_model.generate_milsteinCIR(50,time)[-1])
    

In [82]:
print('Actual: ',exp_stockprice(calibrated_model,50,4))
print('Euler: ', np.mean(sp_euler))
print('PC: ', np.mean(sp_pc))
print('Milstein: ', np.mean(sp_mil))

Actual:  132.86035099173992
Euler:  133.19208385411977
PC:  132.80572110639673
Milstein:  133.15432029695475


PC is the best discretization method among the three discussed

## Appendix

#### 1. OLS

In [14]:
calibrate_dataOLS(df, estimate_metrics=True)

  0%|          | 0/100 [00:00<?, ?it/s]c:\Users\vigne\anaconda3\lib\site-packages\scipy\stats\morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
 32%|███▏      | 32/100 [00:00<00:00, 317.01it/s]

AD Test: Error looks normal (fail to reject H0)
Shapiro Test: Sample looks Gaussian (fail to reject H0)
Mean of Errors:  -1.5359973561319768e-06
Covariance matrix for parameter estimates:  [[2.65193527e+01 2.77841645e-01]
 [2.77841645e-01 3.47790750e-03]]

AD Test: Error does not look normal (reject H0)
Shapiro Test: Sample does not look Gaussian (reject H0)
Mean of Errors:  -1.8721418762017657e-06
Covariance matrix for parameter estimates:  [[2.74432003e+01 2.90586286e-01]
 [2.90586286e-01 3.65448495e-03]]

AD Test: Error looks normal (fail to reject H0)
Shapiro Test: Sample looks Gaussian (fail to reject H0)
Mean of Errors:  9.386730496298689e-07
Covariance matrix for parameter estimates:  [[2.72544256e+01 2.90767829e-01]
 [2.90767829e-01 3.66474907e-03]]

AD Test: Error looks normal (fail to reject H0)
Shapiro Test: Sample looks Gaussian (fail to reject H0)
Mean of Errors:  8.233685584149347e-07
Covariance matrix for parameter estimates:  [[2.68906181e+01 2.82799022e-01]
 [2.8279902

100%|██████████| 100/100 [00:00<00:00, 347.57it/s]

AD Test: Error looks normal (fail to reject H0)
Shapiro Test: Sample looks Gaussian (fail to reject H0)
Mean of Errors:  -4.777094987892468e-07
Covariance matrix for parameter estimates:  [[2.79750122e+01 2.95307303e-01]
 [2.95307303e-01 3.68788053e-03]]

AD Test: Error looks normal (fail to reject H0)
Shapiro Test: Sample looks Gaussian (fail to reject H0)
Mean of Errors:  1.6654138362120178e-06
Covariance matrix for parameter estimates:  [[2.75879940e+01 2.80908977e-01]
 [2.80908977e-01 3.40894052e-03]]

AD Test: Error looks normal (fail to reject H0)
Shapiro Test: Sample looks Gaussian (fail to reject H0)
Mean of Errors:  5.609315272417722e-07
Covariance matrix for parameter estimates:  [[2.69431726e+01 2.83226408e-01]
 [2.83226408e-01 3.54472656e-03]]

AD Test: Error looks normal (fail to reject H0)
Shapiro Test: Sample looks Gaussian (fail to reject H0)
Mean of Errors:  1.8054716502485332e-06
Covariance matrix for parameter estimates:  [[2.70538138e+01 2.84260948e-01]
 [2.84260948

(0.9098468894490687, 135.09397038781802, 0.399912442590426)

In [15]:
for i in tqdm(range(100)):
    Y = (df[str(i)].diff())/np.sqrt(df[str(i)])
    x1 = df['T'].diff()/np.sqrt(df[str(i)])
    x2 = -np.sqrt(df[str(i)])*(df['T'].diff())

    Y.drop(0,inplace=True)
    x1.drop(0,inplace=True)
    x2.drop(0,inplace=True)

    X = np.array((x1,x2)).T
    lr = sm.OLS(Y, X).fit()
    print(lr.summary())

 25%|██▌       | 25/100 [00:00<00:00, 127.86it/s]

                                 OLS Regression Results                                
Dep. Variable:                      0   R-squared (uncentered):                   0.094
Model:                            OLS   Adj. R-squared (uncentered):              0.094
Method:                 Least Squares   F-statistic:                              387.8
Date:                Mon, 31 Oct 2022   Prob (F-statistic):                   5.37e-161
Time:                        22:50:24   Log-Likelihood:                          25511.
No. Observations:                7499   AIC:                                 -5.102e+04
Df Residuals:                    7497   BIC:                                 -5.100e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

 39%|███▉      | 39/100 [00:00<00:00, 132.23it/s]

                                 OLS Regression Results                                
Dep. Variable:                     26   R-squared (uncentered):                   0.091
Model:                            OLS   Adj. R-squared (uncentered):              0.091
Method:                 Least Squares   F-statistic:                              376.1
Date:                Mon, 31 Oct 2022   Prob (F-statistic):                   2.18e-156
Time:                        22:50:24   Log-Likelihood:                          25551.
No. Observations:                7499   AIC:                                 -5.110e+04
Df Residuals:                    7497   BIC:                                 -5.109e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

 70%|███████   | 70/100 [00:00<00:00, 141.16it/s]

                                 OLS Regression Results                                
Dep. Variable:                     53   R-squared (uncentered):                   0.091
Model:                            OLS   Adj. R-squared (uncentered):              0.091
Method:                 Least Squares   F-statistic:                              376.6
Date:                Mon, 31 Oct 2022   Prob (F-statistic):                   1.46e-156
Time:                        22:50:24   Log-Likelihood:                          25578.
No. Observations:                7499   AIC:                                 -5.115e+04
Df Residuals:                    7497   BIC:                                 -5.114e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

100%|██████████| 100/100 [00:00<00:00, 140.69it/s]

                                 OLS Regression Results                                
Dep. Variable:                     81   R-squared (uncentered):                   0.100
Model:                            OLS   Adj. R-squared (uncentered):              0.100
Method:                 Least Squares   F-statistic:                              416.7
Date:                Mon, 31 Oct 2022   Prob (F-statistic):                   2.54e-172
Time:                        22:50:24   Log-Likelihood:                          25651.
No. Observations:                7499   AIC:                                 -5.130e+04
Df Residuals:                    7497   BIC:                                 -5.128e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### 2. ML 

In [17]:
a,b,sigma

(0.9100101993332262, 135.09418157986403, 0.39990734424709956)

#### 3. Solvers

In [42]:
calibrate_dataMLE(df, method = "Nelder-Mead")

100%|██████████| 100/100 [00:22<00:00,  4.42it/s]

No of iter:  65.02


(0.9100101993332262, 135.09418157986403, 0.39990734424709956)

In [43]:
negloglik((0.9100101993332262, 135.09418157986403, 0.39990734424709956),T,df)[1:].mean()

c:\Users\vigne\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


-12080.107425385959

In [44]:
calibrate_dataMLE(df, method = "BFGS")

100%|██████████| 100/100 [00:40<00:00,  2.47it/s]

No of iter:  2.58


(0.9098549658090235, 135.09396824764636, 0.39990949958243)

In [45]:
negloglik((0.9098549658090235, 135.09396824764636, 0.39990949958243),T,df)[1:].mean()

c:\Users\vigne\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


-12080.212661632959

In [46]:
calibrate_dataMLE(df, method = "L-BFGS-B")

100%|██████████| 100/100 [00:06<00:00, 14.73it/s]

No of iter:  0.99


(0.9069713800379191, 135.06920819671774, 0.3425973787638769)

In [47]:
negloglik((0.9069713800379191, 135.06920819671774, 0.3425973787638769),T,df)[1:].mean()

c:\Users\vigne\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


-13096.108099293984